In [160]:
import re
import pandas as pd
import win32com.client
import pickle
import os
import glob
import email

In [161]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

## Finding the Email folder you wish to scrape

In [162]:
for i in range(25):
    try:
        box = outlook.GetDefaultFolder(i)
        name = box.Name
        print(i, name)
    except:
        pass

3 Deleted Items
4 Outbox
5 Sent Items
6 Inbox
9 Calendar
10 Contacts
11 Journal
12 Notes
13 Tasks
14 Reminders
15 Reminders
16 Drafts
17 Conflicts
19 Conflicts
20 Sync Issues
21 Local Failures
22 Server Failures
23 Junk Email


## Scraping Email Content Directly

In [3]:
inbox = outlook.GetDefaultFolder(6) # "6" refers to the index of a folder - in this case,
                                    # the inbox. You can change that number to reference
                                    # any other folder
messages = inbox.Items
message = messages.GetLast()
body_content = message.body

# print (body_content) #If you want to check the body of the most recent message

In [12]:
#note that this cell searches the bdy of an email
#you must change if you want to view the actual subject  / sender details

emails = []
for message in messages:
    if 'block payment' in message.body.lower(): 
        emails.append( (message.body) )

In [22]:
emails[0] #prints the most recent email from your search

" \r\n\r\n \r\n\r\nFrom: Neil Richardson <Neil.Richardson@triton-global.com> \r\nSent: 15 September 2016 16:59\r\nTo: Credit Control <creditcontrol@Castelmga.com>\r\nCc: Tom Steibelt <tom.steibelt@Castelmga.com>; ARP Claims <claims@Altituderiskpartners.com>\r\nSubject: Block payment USD 17,040.73 - Payment from Triton Global Limited Castel (ARP) - 15Sept2016 - 7 Payments\r\n\r\n \r\n\r\nGood afternoon\r\n\r\n \r\n\r\nPlease find below details of a block payment to Castel with a payment date of 15/09/2016\r\n\r\n \r\n\r\nTriton Ref\r\n\r\nInsured\r\n\r\nType\r\n\r\nComment\r\n\r\nPayment Date\r\n\r\nCUR\r\n\r\nTotal\r\n\r\nPayee\r\n\r\nCM1795\r\n\r\nBRITISH AIRWAYS\r\n\r\nClaim\r\n\r\nARP ref: 10129000116\r\n\r\nUCR: B0509AA0008160003\r\n\r\n15/09/16\r\n\r\nUSD\r\n\r\n1,500.00\r\n\r\nCastel\r\n\r\nCM1791\r\n\r\nBRITISH AIRWAYS\r\n\r\nClaim\r\n\r\nARP ref: 10080100115\r\n\r\n \r\n\r\nUCR: B0509AA000816L003\r\n\r\n15/09/16\r\n\r\nUSD\r\n\r\n2,090.91\r\n\r\nCastel\r\n\r\nCM1806\r\n\r\nKLM 

In [24]:
email_concat = pd.DataFrame()
for i in emails:
    dic={}
    dic['Claim Ref'] = re.findall('([C][M][0-9]+)' , i[i.find('CM')-4:])
    #dic['Insured'] = re.findall('[C][M][0-9]+[\\rn]+([A-Za-z0-9\s])+[\\rn]+[C][l][a][i][m]' , i[i.find('CM')-4:])
    dic['UCR'] = re.findall('[U][C][R][: ]+([BC ][BC 0-9][ 0-9A-Za-z]+)' , i[i.find('CM')-4:])
    dic['ARP Ref'] = re.findall('[A][R][P][ ][r][e][f][: ]+([& 0-9A-Za-z]+)' , i[i.find('CM')-4:])
    dic['Settlement Currency'] = re.findall('([U][S][D]|[G][B][P]|[E][U][R]|[J][P][Y]|[Z][A][R]|[A][U][D]|[C][A][D]|[S][G][D])' , i[i.find('CM')-4:])
    dic['Amount'] = [i.strip() for i in re.findall('([\s\w\d\s][\s\d\w][,\w\d\s][0-9\w\d\s][0-9][0-9][.][0-9][0-9])' , i[i.find('CM')-4:]) ]
    dic['Payment Date'] =  re.findall('[0-9][0-9][/][0-9][0-9][/][0-9][0-9]' , i[i.find('CM')-4:])
    
    
    #scraping the body of an email directly into a dataframe, below checks that each array is the same length
    count = []
    for x in dic.keys():
        count.append( len(dic[x]))
    if len(set(count)) == 1:
        print ('ok')
    else:
        print ('Problem' + str ( re.findall( '[S][e][n][t][:][ ]([ 0-9 A-Z a-z :]+)' , i ) ) \
               + str ( re.findall( '[S][u][b][j][e][c][t][:][ ]([0-9A-Za-z :,.]+)' , i ) ) )
        
    
    email = pd.DataFrame(data =  dic)
    email_concat = pd.concat([email_concat , email]) 

    print(dic)

ok
{'Claim Ref': ['CM1795', 'CM1791', 'CM1806', 'CM1802', 'CM1803', 'CM1807', 'CM1820'], 'UCR': ['B0509AA0008160003', 'B0509AA000816L003', 'B 0509 XAA024815ABB', 'B0509AA001216B001           ', 'B0509AA247916001', 'B 0509 XAA024815AAD', 'B 0801 04476A15AAA'], 'ARP Ref': ['10129000116', '10080100115', '10057000115', '10080100115', '10129000116', '10057000115', '10085300115'], 'Settlement Currency': ['USD', 'USD', 'USD', 'USD', 'USD', 'USD', 'USD'], 'Amount': ['1,500.00', '2,090.91', '10,000.00', '522.73', '375.00', '2,500.00', '52.09'], 'Payment Date': ['15/09/16', '15/09/16', '15/09/16', '15/09/16', '15/09/16', '15/09/16', '15/09/16']}
ok
{'Claim Ref': ['CM1790', 'CM1461', 'CM1793', 'CM1794'], 'UCR': ['B080118951A15ADA ', 'B5064A07000363', 'B0509AA001516C001', 'B0509AA001516CL01 '], 'ARP Ref': ['10057500115 and 10106200115', '10090200115', '10129000116', '10080100115'], 'Settlement Currency': ['USD', 'USD', 'USD', 'USD'], 'Amount': ['70,000.00', '68.81', '250.00', '348.49'], 'Payment D

## Savings Attachments from an email

In [238]:
#As mentioned above, we are now scraping the subject of emails.

for i in messages:
    if len (re.findall('[S][u][b][j][e][c][t][:][ ][Aa]' , message.subject)) > 0:
            print(message.body)

In [4]:
get_path = r"C:\Users\Consultant\Documents\Castel\Roland\email_downloads"

#following loop scrapes and downloads all of the attachments based on the Subject containing AON.

emails2 = []
for message in messages:
    if len(re.findall('[A][o][n]' , message.subject)) > 0 : 
        emails2.append( (message.body))
        print('appended' + str(message.subject))
        attachments = message.Attachments
        print('Length of attachments '+len(attachments))
        counter = len(attachments)
        countercheck = 0
        for i in range(0, len(attachments)+2):
            try:
                attachment = attachments.Item(i)
                attachment.SaveASFile(os.path.join(get_path, attachment.FileName[-30:])) #Saves to the attachment to current folder
                print(i)
                print (attachment)
                countercheck+=1
            except:
                print
                print(str(i) +' didnt work ')
                pass
        if counter != countercheck:
            print ('PROBLEM WITH THIS EMAIL !' + str(message.subject))

appended
7
0 didnt work 
1
FW: URGENT CASH CALL :  HULL Settlement  - Binding Authority B087517C0CN5001 (2017 year) New claim: AV152758 Parachuting Aircraft Ltd   Double fatal accident 13/5/2018    Our ref: 5907556 (URGENT)
2
RE: Remittance Advice
3
FW: GRiDS: Disb: Castel Underwriting Agencies Limited [(69,760.78) USD] 11-Jul-2018
4
GRiDS: Disb: Castel Underwriting Agencies Limited [(754,256.78) USD] 10-Jul-2018|GRiDS: Disb: Castel Underwriting Agencies Limited [(171,104.00) USD] 10-Jul-2018|GRiDS: Disb: Castel Underwriting Agencies Limited [(45,153.91) EUR] 10-Jul-2018|GRiDS: Disb: 
5
FW: GRiDS: Disb: Castel Underwriting Agencies Limited [(171,242.10) USD] 09-Jul-2018
6
GRiDS: Disb: Castel Underwriting Agencies Limited [(18,956.54) EUR] 06-Jul-2018|GRiDS: Disb: Castel Underwriting Agencies Limited [(25,740.31) GBP] 06-Jul-2018
7
GRiDS: Disb: Castel Underwriting Agencies Limited [(2,391.37) GBP] 05-Jul-2018|GRiDS: Disb: Castel Underwriting Agencies Limited [(239,503.18) USD] 05-Jul-20

## Opening up .msg files, then downloading their attachments (repeatable) should put in a function

In [11]:
## Once these attachments have been downloaded
#we can open them up as .msg files and then keep reopening them until we get the attachment we want.  

level_two_path = get_path + "\Level 2"
for i in level_one_emails[1:]:
    msg = outlook.OpenSharedItem(i)
    count_attachments = msg.Attachments.Count
    if count_attachments > 0:
        for item in range(count_attachments):
            attachment = msg.Attachments.Item(item + 1)
            print (msg.Attachments.Item(item + 1).Filename)
            attachment.SaveASFile(os.path.join(level_two_path, attachment.FileName[-100:]))

com_error: (-2147352567, 'Exception occurred.', (4096, 'Microsoft Outlook', 'Invalid path or URL.', None, 0, -2147352567), None)

In [11]:
level_three_path = r"C:\Users\Consultant\Documents\Castel\Roland\email_downloads\Level 2\Level 3"
for i in level_two_emails[1:]:
    msg = outlook.OpenSharedItem(i)
    count_attachments = msg.Attachments.Count
    if count_attachments > 0:
        for item in range(count_attachments):
            attachment = msg.Attachments.Item(item + 1)
            print (msg.Attachments.Item(item + 1).Filename)
            attachment.SaveASFile(os.path.join(level_three_path, attachment.FileName[-100:]))

DisbBackup_200654701_20180706103118840.xlsx
DisbBackup_200654538_20180705100418300.xlsx
DisbBackup_200654546_20180705100407843.xlsx
DisbBackup_200654697_20180706103104646.xlsx
DisbBackup_200654840_20180710100410788.xlsx
DisbBackup_200654839_20180710100413486.xlsx
DisbBackup_200654841_20180710100415793.xlsx


## Opening up .eml files and downloading their attachments

In [158]:
#This doesnt work for .msg attachments for some reason
path = r"C:\Users\Consultant\Documents\Castel\Lineslips\Tom\emails"

listing = os.listdir(path)

for file in listing:
    
    if str.lower(file[-3:])=="eml":
        msg = email.message_from_file(open(path + '\\'+ file))
        attachments = msg.get_payload()
        
        for attachment in attachments[1:]: #don't know why but there seems to be an empty attachment at the beginning of each payload
            
            try:
                filename = attachment.get_filename()
                #print(filename)
                f = open(os.path.join(path + '\\Level 2\\'+filename), 'wb').write(attachment.get_payload(decode=True))
                #f.close()
            
            except Exception as detail:
                print('Error: {}'.format(file))
                print (detail)


Error: CAD 106875.eml
must be str, not NoneType
